In [1]:
import tensorflow as tf
import numpy as np
import os
import sys
import datetime

In [2]:
data_file_name = 'face_data.npz'
test_file_name = 'test_data.npz'

batch_size = 128

def init_weights(shape, name):
    """
    initialize weight
    shape: shape of weight
    name: weight name
    각각의 Filter를 만들어주는 함수
    """
    with tf.device('/gpu:0'):
        return tf.Variable(tf.random_normal(shape, stddev = 0.005), name = name)

def init_bias(shape, name):
    """
    initialize bias
    bias를 만들어주는 함수
    """
    with tf.device('/gpu:0'):
        return tf.Variable(tf.constant(0.1, shape = shape))

def model(X, w1, w11, w2, w21, w3, w4, w5, wo, b4,  b5, bo, p_keep_conv, p_keep_hidden):
    """
    create model
    X: first input
    w1, w2, w3, wo: weights for each layer
    b2, b3, bo: bias for each layer
    p_keep_conv, p_keep_hidden: constant for dropout

    모델생성 함수.
    총 3개의 Conv layers와 3개의 Max-pool layers, 2개의 FC layers로 구성
    Conv1 - Pooling1 - Conv2 - Pooling2 - Conv3 - Pooling3 - FC1 - FC2
    로 구성되어있고 각 풀링 이후와 첫번째 FC 이후에 dropout을 적용
    """
    
    with tf.device('/gpu:0'):
        with tf.name_scope("layer1"):
            # First Conv layer. l1a shape = (?, 128, 128, 32)
            l1a = tf.nn.relu(tf.nn.conv2d(X, w1, strides = [1, 1, 1, 1], padding = 'SAME'))
            l1 = tf.nn.max_pool(l1a, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')  
            # l11 shape = (?, 64, 64, 32)
            
#         with tf.name_scope("layer1b"):
#             l1b = tf.nn.relu(tf.nn.conv2d(l11, w11, strides = [1, 1, 1, 1], padding = 'SAME'))
#             l1 = tf.nn.max_pool(l1b, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')        
#             l1 = tf.nn.dropout(l1, p_keep_conv)
#             # l1 shape = (?, 32, 32, 64)

        with tf.name_scope("layer2"):
            # l2a shape = (?, 64, 64, 64)
            l2a = tf.nn.relu(tf.nn.conv2d(l1, w2, strides = [1, 1, 1, 1], padding = 'SAME'))
            l2 = tf.nn.max_pool(l2a, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
            # l21 shape = (?, 16, 16, 128)
            
#         with tf.name_scope("layer2b"):
#             l2b = tf.nn.relu(tf.nn.conv2d(l21, w21, strides = [1, 1, 1, 1], padding = 'SAME'))
#             l2 = tf.nn.max_pool(l2b, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
#             l2 = tf.nn.dropout(l2, p_keep_conv)
#             # l2 shape = (?, 8, 8, 256)
            
        with tf.name_scope("layer3"):
            l3a = tf.nn.relu(tf.nn.conv2d(l2, w3, strides = [1, 1, 1, 1], padding = 'SAME'))
            l3 = tf.nn.max_pool(l3a, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
            l3 = tf.nn.dropout(l3, p_keep_conv)
            # l3 shape = (?, 4, 4, 512)

        with tf.name_scope("layer4"):
            l4a = tf.nn.relu(tf.nn.conv2d(l3, w4, strides = [1, 1, 1, 1], padding = 'SAME') + b4)
            # l2 shape = (?, 2, 2, 1024)
            l4 = tf.nn.max_pool(l4a, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
            l4 = tf.reshape(l4, [-1, w5.get_shape().as_list()[0]])
            l4 = tf.nn.dropout(l4, p_keep_conv)

        with tf.name_scope("layer5"):
            # reshape to ( , 2*2*1024)
            l5 = tf.nn.relu(tf.matmul(l4, w5) + b5)
            l5 = tf.nn.dropout(l5, p_keep_hidden)
            out = tf.matmul(l5, wo) + bo
            return out

In [3]:
# 데이터를 불러온다

if os.path.isfile(data_file_name):
    file = np.load(data_file_name)
    data = file['data']
    labels = file['age_labels']
else:
    print("No data file.")
    sys.exit()
    
if os.path.isfile(test_file_name):
    file = np.load(test_file_name)
    td = file['data']
    tl = file['age_labels']
else:
    print("No test file")
    sys.exit()

print("data load success")
print(labels.shape)

data load success
(1785, 3)


In [4]:
tf.reset_default_graph()

# data, labels 를 위한 placeholder
X = tf.placeholder("float", [None, 128, 128, 1], name = "X")
Y = tf.placeholder("float", [None, 3], name = "Y")

# 각각의 weight와 bias를 초기화
w1 = init_weights([7, 7, 1, 32], "weight1") 
w11 = init_weights([5, 5, 32, 64], "weight11") 
w2 = init_weights([5, 5, 32, 64], "weight2")
w21 = init_weights([5, 5, 128, 256], "weight21") 
w3 = init_weights([5, 5, 64, 128], "weight3")
w4 = init_weights([3, 3, 128, 256], "weight4")
w5 = init_weights([8*8*256, 625], "weight5")
wo = init_weights([625, 3], "weight_out")

b4 = init_bias([256], "bias4")
b5 = init_bias([625], "bias5")
bo = init_bias([3], "bias_out")

# tensorboard에 weights와 bias histogram 등록
tf.summary.histogram("weight1_summ", w1)
tf.summary.histogram("weight2_summ", w2)
tf.summary.histogram("weight3_summ", w3)
tf.summary.histogram("weight4_summ", w4)
tf.summary.histogram("bias4_summ", b4)
tf.summary.histogram("weight5_summ", w5)
tf.summary.histogram("bias5_summ", b5)
tf.summary.histogram("weight_out_summ", wo)
tf.summary.histogram("bias_out_summ", bo)

tf.summary.histogram("weight11_summ", w11)
tf.summary.histogram("weight21_summ", w21)

# dropout 위한 placeholder
p_keep_conv = tf.placeholder("float", name = "p_keep_conv")
p_keep_hidden = tf.placeholder("float", name = "p_keep_hidden")

# 모델생성
M = model(X, w1, w11, w2, w21, w3, w4, w5, wo, b4, b5, bo, p_keep_conv, p_keep_hidden)

# cost와 accuarcy를 계산해주는 함수 설정
with tf.name_scope("cost"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = M, labels = Y))
    train_op = tf.train.RMSPropOptimizer(0.00001, 0.9).minimize(cost)
    tf.summary.scalar("cost", cost)

with tf.name_scope("accuarcy"):
    correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(M, 1))
    acc_op = tf.reduce_mean(tf.cast(correct_pred, "float"))
    tf.summary.scalar("accuracy", acc_op)

# 모델 저장을 위해 collection에 weight와 bias모음
tf.add_to_collection('vars', w1)
tf.add_to_collection('vars', w11)
tf.add_to_collection('vars', w2)
tf.add_to_collection('vars', w21)
tf.add_to_collection('vars', w3)
tf.add_to_collection('vars', w4)
tf.add_to_collection('vars', w5)
tf.add_to_collection('vars', b5)
tf.add_to_collection('vars', wo)
tf.add_to_collection('vars', bo)

In [5]:
# 모델 저장 경로 설정
save_path = "./models/"
if not os.path.exists(save_path):
    os.makedirs(save_path)

# 세션시작
with tf.Session(config = tf.ConfigProto(gpu_options = tf.GPUOptions(allow_growth = True))) as sess:
    
    # 모델 저장
    writer = tf.summary.FileWriter('./logs/face_a', sess.graph)
    merged = tf.summary.merge_all()
    
    tf.global_variables_initializer().run()
    
    saver = tf.train.Saver()
    
    print(datetime.datetime.now().time())
    
    # epoch=100000, batch size=128로 모델 학습
    for i in range(100000):
        training_batch = zip(range(0, len(data), batch_size), range(batch_size, len(data)+1, batch_size))
        for start, end in training_batch:
            sess.run(train_op, feed_dict = {X: data[start:end], Y: labels[start:end],
                                           p_keep_conv: 0.9, p_keep_hidden: 0.6})
        
        summary, acc = sess.run([merged, acc_op], feed_dict = {X: td, Y: tl, p_keep_conv: 1.0, p_keep_hidden: 1.0})
        
        writer.add_summary(summary, i)
        print(i, acc)
    
    saved_path = saver.save(sess, os.path.join(save_path, 'face_recog_a.ckpt'))
    print("Checkpoint saved at ", saved_path)
    print(datetime.datetime.now().time())
    sess.close()

17:30:44.730530
0 0.271084
1 0.271084
2 0.271084
3 0.271084
4 0.271084
5 0.271084
6 0.271084
7 0.271084
8 0.271084
9 0.271084
10 0.271084
11 0.271084
12 0.271084
13 0.271084
14 0.271084
15 0.271084
16 0.271084
17 0.271084
18 0.271084
19 0.271084
20 0.271084
21 0.271084
22 0.271084
23 0.271084
24 0.271084
25 0.271084
26 0.271084
27 0.271084
28 0.271084
29 0.271084
30 0.271084
31 0.271084
32 0.271084
33 0.271084
34 0.271084
35 0.271084
36 0.271084
37 0.271084
38 0.271084
39 0.271084
40 0.271084
41 0.271084
42 0.271084
43 0.271084
44 0.271084
45 0.271084
46 0.271084
47 0.271084
48 0.271084
49 0.271084
50 0.271084
51 0.271084
52 0.271084
53 0.271084
54 0.271084
55 0.271084
56 0.271084
57 0.271084
58 0.271084
59 0.271084
60 0.271084
61 0.271084
62 0.271084
63 0.271084
64 0.271084
65 0.271084
66 0.271084
67 0.271084
68 0.271084
69 0.271084
70 0.271084
71 0.271084
72 0.271084
73 0.271084
74 0.271084
75 0.271084
76 0.271084
77 0.271084
78 0.271084
79 0.271084
80 0.271084
81 0.271084
82 0.27108